# Test For Data Analyst Position at JobTech

## Basic data retrieval (pyspark)

### Environmental parameters
- Set environment to run Pyspark
- Import dependent packages & functions for the notebook

In [1]:
import os
os.environ["JAVA_HOME"] = "C:/Program Files/Java/jre1.8.0_311"
os.environ["SPARK_HOME"] = "C:/Spark/spark-3.2.0-bin-hadoop3.2"

path_drive = 'C:/Users/adam2/Documents/data_analyst_test'

import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f

#### Start Pyspark session and import "test_data.txt" as a PySpark table

*Note that the provided "test_data.xlsx" was converted to tab-separated format on MS Excel*

In [2]:
sc = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
df = sc.read.csv(f"{path_drive}/test_data.txt", inferSchema=True, header=True, sep="\t")

In [4]:
print(df.count(), len(df.columns))

3000 5


In [5]:
print(df.printSchema())

root
 |-- job_id: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- date_posted: string (nullable = true)
 |-- company_name: string (nullable = true)

None


In [6]:
df \
    .orderBy('job_id') \
    .show(5)

+--------------------+--------------------+--------------------+-----------+--------------+
|              job_id|           job_title|     job_description|date_posted|  company_name|
+--------------------+--------------------+--------------------+-----------+--------------+
|0004462a2bb5b651d...|3 Months Temp Sup...|Summary:<br/><br/...| 2020-11-12|RecruitExpress|
|001296f2dad620884...|Specialist (Advan...|We are looking fo...| 2020-11-26|  Confidential|
|002b9c2f0f3c5e71c...|BIM Coordinator (...|&bull; Main Con&b...| 2020-11-22|  Confidential|
|002f1f6b0a375933c...|IT Support Engine...|Industry: Bunkeri...| 2020-11-11|  Confidential|
|004aaba90989955da...|Warehouse Assista...|Provide administr...| 2020-11-23|  Confidential|
+--------------------+--------------------+--------------------+-----------+--------------+
only showing top 5 rows



#### Data Cleaning
1. Cast date_posted to data format
2. Identify duplicates
3. Remove whitespace in string variables
4. Identify missing values

*1. Cast date_posted as date*

In [7]:
df = df \
    .withColumn('date_posted', f.to_date(f.col('date_posted'), 'yyyy-MM-dd') \
                .alias('date'))

*2. Drop duplicates*

In [8]:
df = df.dropDuplicates(["job_id"])
print(df.count(), len(df.columns))

3000 5


*3. Remove whitespace in string variables*

In [9]:
str_cols = [item[0] for item in df.dtypes if item[1] == "string"]
str_cols

['job_id', 'job_title', 'job_description', 'company_name']

In [10]:
for col in str_cols:
    df = df \
        .withColumn(col, f.trim(df[col]))

In [11]:
df \
    .orderBy('job_id') \
    .show(5)

+--------------------+--------------------+--------------------+-----------+--------------+
|              job_id|           job_title|     job_description|date_posted|  company_name|
+--------------------+--------------------+--------------------+-----------+--------------+
|0004462a2bb5b651d...|3 Months Temp Sup...|Summary:<br/><br/...| 2020-11-12|RecruitExpress|
|001296f2dad620884...|Specialist (Advan...|We are looking fo...| 2020-11-26|  Confidential|
|002b9c2f0f3c5e71c...|BIM Coordinator (...|&bull; Main Con&b...| 2020-11-22|  Confidential|
|002f1f6b0a375933c...|IT Support Engine...|Industry: Bunkeri...| 2020-11-11|  Confidential|
|004aaba90989955da...|Warehouse Assista...|Provide administr...| 2020-11-23|  Confidential|
+--------------------+--------------------+--------------------+-----------+--------------+
only showing top 5 rows



*4. Identify any missing values*

In [12]:
df \
    .filter('job_title IS NULL') \
    .count()

0

In [13]:
valid_count = []

for col in str_cols:
    out = df \
        .withColumn(col, f.when(df[col] != 'empty-value', df[col])).count()
    valid_count.append(out)

print(valid_count)

[3000, 3000, 3000, 3000]


#### Preliminary data exploration

*1. total number of jobs posted by DBS*

In [14]:
df \
    .filter(f.col('company_name') \
            .startswith('DBS')) \
    .orderBy('job_id') \
    .show(5)

+--------------------+--------------------+--------------------+-----------+----------------+
|              job_id|           job_title|     job_description|date_posted|    company_name|
+--------------------+--------------------+--------------------+-----------+----------------+
|0b75a89c3d37660e4...|Data Engineer,Gro...|!*!Business Funct...| 2020-11-20|             DBS|
|1cca7709a8a70f00f...|AVP, Digital Prod...|AVP, Digital Prod...| 2020-11-06|DBS Bank Limited|
|2135bd7a1b17a75c5...|Senior Associate ...|!*!Business Funct...| 2020-11-25|             DBS|
|311e0ade8d1162e8f...|Direct Contract, ...|Direct Contract, ...| 2020-11-25|DBS Bank Limited|
|322659e6a05e11d9d...|AVP/ Senior Assoc...|AVP/ Senior Assoc...| 2020-11-23|DBS Bank Limited|
+--------------------+--------------------+--------------------+-----------+----------------+
only showing top 5 rows



In [15]:
dbs_job_count = df \
    .filter(f.col('company_name') \
            .startswith('DBS')) \
    .orderBy('job_id') \
    .count()

print(f'Total number of jobs posted by DBS bank: {dbs_job_count}')

Total number of jobs posted by DBS bank: 21


*2. retrive all jobs posted by companies containing “Pte. Ltd.”*

In [16]:
df \
    .filter(f.col('company_name') \
            .contains('Pte. Ltd')) \
    .orderBy('job_title') \
    .show()

+--------------------+--------------------+--------------------+-----------+--------------------+
|              job_id|           job_title|     job_description|date_posted|        company_name|
+--------------------+--------------------+--------------------+-----------+--------------------+
|1ce57ba094a5d8253...|Accounts cum Admi...|Job Responsibilit...| 2020-11-02|One Tax CM Pte. L...|
|35998862bfc579334...|           Bartender|We are looking fo...| 2020-11-15|J Capital Plus Pt...|
|428389f7d5c4a6e7e...|Full-time Persona...|Passion+attitudeT...| 2020-11-19|PURE Internationa...|
|efe7dad2065d007c7...|Sales Team Leader...|ASIA&#8217;S LEAD...| 2020-11-05|PURE Internationa...|
+--------------------+--------------------+--------------------+-----------+--------------------+



*3. count of job postings by date*

In [17]:
df \
    .groupby('date_posted') \
    .count() \
    .show()

+-----------+-----+
|date_posted|count|
+-----------+-----+
| 2020-12-18|    3|
| 2020-12-17|    3|
| 2020-12-30|    1|
| 2020-11-06|   85|
| 2020-12-13|    3|
| 2020-12-23|    1|
| 2020-12-12|    2|
| 2020-12-09|    3|
| 2020-11-27|  134|
| 2020-11-16|   39|
| 2020-11-09|  146|
| 2020-12-16|    2|
| 2020-11-19|  203|
| 2020-12-20|    1|
| 2020-11-05|   83|
| 2020-11-10|  112|
| 2020-11-01|   18|
| 2020-11-02|   35|
| 2020-11-28|    1|
| 2020-11-17|  155|
+-----------+-----+
only showing top 20 rows



*4. count of job postings by month*

In [18]:
df = df\
    .withColumn('month', f.month(f.col('date_posted'))
                .alias('month'))

In [19]:
df\
    .groupby('month')\
    .count()\
    .show()

+-----+-----+
|month|count|
+-----+-----+
|   12|   44|
|   11| 2956|
+-----+-----+

